<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from node2vec import Node2Vec
from node2vec.edges import HadamardEmbedder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition, datasets
from sklearn.manifold import TSNE
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
path = './data/'
emb_data_train = pd.read_csv('./data/node2vec_v2_train.csv.gz',
                             compression='gzip',
                             sep='\t')
emb_data_test = pd.read_csv('./data/node2vec_v2_test.csv.gz',
                            compression='gzip',
                            sep='\t')
emb_data_train = emb_data_train.drop(columns=['Unnamed: 0'])
emb_data_test = emb_data_test.drop(columns=['Unnamed: 0'])
emb_data_train.head()

,is_dorm,is_year,year_diff,from_high_school,to_high_school,from_major,to_major,is_faculty,is_gender,d0,...,d59,d6,d60,d61,d62,d63,d7,d8,d9,label
0,0,0,1.000000,17819,50093,265,294,1,1,0.586327,...,0.080709,0.663903,0.521325,-1.218457,0.572862,8.867662,0.098672,0.072862,1.136375,0
1,0,1,0.000000,10070,24562,51,60,1,1,1.556291,...,2.682853,-0.022940,-0.371618,-0.987022,1.831474,-0.448014,1.167990,2.401762,0.431279,0
2,0,1,0.000000,1544,6122,14,39,1,1,-0.649554,...,6.857399,-0.348564,2.753245,-0.207652,1.892210,-0.977367,11.321891,2.510515,2.910005,1
3,0,0,2.196375,1894,50410,0,40,0,0,3.757138,...,5.611234,-1.006211,-1.872412,-0.861397,0.114388,-1.096610,-4.892152,-3.294941,0.133597,0
4,0,1,0.000000,3535,21037,238,271,1,1,1.290842,...,0.824280,0.429348,2.083774,0.281935,-1.777484,0.258835,0.234619,-0.287869,2.079238,0


In [3]:
def LogReg(data, test, params={}, cols=[], model=False):
    cols.append('label')
    X = data.drop(columns=cols).values
    y = data['label'].values
    X_test = test.drop(columns=cols).values
    y_test = test['label'].values
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler.fit(X)
    X = scaler.transform(X)
    print('X:', X.shape)
    model = LogisticRegression(**params).fit(X, y)
    yhat = model.predict(X_test)
    print(confusion_matrix(y_test, yhat))
    print(classification_report(y_test, yhat))
    print('ROC_AUC_SCORE:',
          roc_auc_score(y_test, model.decision_function(X_test)))
    return model

In [4]:
def emb_data(cols):

    emb_data_train = pd.read_csv('./data/node2vec_v2_train.csv.gz',
                                 compression='gzip',
                                 sep='\t')
    emb_data_test = pd.read_csv('./data/node2vec_v2_test.csv.gz',
                                compression='gzip',
                                sep='\t')
    emb_data_train = emb_data_train.drop(columns=['Unnamed: 0'])
    emb_data_test = emb_data_test.drop(columns=['Unnamed: 0'])
    params = {
        'C': 150,  #100,
        'class_weight': None,
        'dual': False,
        'fit_intercept': True,
        'intercept_scaling': 1,
        'l1_ratio': None,
        'max_iter': 100000,
        'multi_class': 'auto',
        'n_jobs': None,
        'penalty': 'l1',
        'random_state': None,
        'solver': 'liblinear',
        'tol': 0.0001,
        'verbose': 1,
        'warm_start': False
    }
    return LogReg(emb_data_train,
                  emb_data_test,
                  params,
                  cols=cols)

In [5]:
'''
cols = ["is_dorm","is_year","year_diff", "from_high_school",
            "to_high_school", "from_major", "to_major", "is_faculty",
            "is_gender",'label']
X =  emb_data_train.drop(columns=cols).values
y = emb_data_train['label'].values

scaler = MinMaxScaler(feature_range = (0,1))
scaler.fit(X)
X = scaler.transform(X)
# Create regularization penalty space
penalty = ['l1', 'l2']

# Create regularization hyperparameter space
C = [0.001,0.1,1,10,100]
# Get solver
solver = ['liblinear']
# Create hyperparameter options
hyperparameters = dict(C=C, penalty=penalty,solver=solver)
# Create grid search using 5-fold cross validation
clf = GridSearchCV(LogisticRegression(max_iter=100000), hyperparameters, cv=5, verbose=True,n_jobs=-1)
# Fit grid search
best_model = clf.fit(X, y)
# View best hyperparameters

print('PARAMS:', best_model.best_estimator_.get_params())
params = best_model.best_estimator_.get_params() 
'''

'\ncols = ["is_dorm","is_year","year_diff", "from_high_school",\n            "to_high_school", "from_major", "to_major", "is_faculty",\n            "is_gender",\'label\']\nX =  emb_data_train.drop(columns=cols).values\ny = emb_data_train[\'label\'].values\n\nscaler = MinMaxScaler(feature_range = (0,1))\nscaler.fit(X)\nX = scaler.transform(X)\n# Create regularization penalty space\npenalty = [\'l1\', \'l2\']\n\n# Create regularization hyperparameter space\nC = [0.001,0.1,1,10,100]\n# Get solver\nsolver = [\'liblinear\']\n# Create hyperparameter options\nhyperparameters = dict(C=C, penalty=penalty,solver=solver)\n# Create grid search using 5-fold cross validation\nclf = GridSearchCV(LogisticRegression(max_iter=100000), hyperparameters, cv=5, verbose=True,n_jobs=-1)\n# Fit grid search\nbest_model = clf.fit(X, y)\n# View best hyperparameters\n\nprint(\'PARAMS:\', best_model.best_estimator_.get_params())\nparams = best_model.best_estimator_.get_params() \n'

In [6]:
params = {
    'C': 150,  #100,
    'class_weight': None,
    'dual': False,
    'fit_intercept': True,
    'intercept_scaling': 1,
    'l1_ratio': None,
    'max_iter': 100000,
    'multi_class': 'auto',
    'n_jobs': None,
    'penalty': 'l1',
    'random_state': None,
    'solver': 'liblinear',
    'tol': 0.0001,
    'verbose': 1,
    'warm_start': False
}

In [54]:
def unseen_emb_data():
    cols = [
        "is_dorm", "is_year", "year_diff", "from_high_school",
        "to_high_school", "from_major", "to_major", "is_faculty", "is_gender",'label'
    ]
    
    unseen = pd.read_csv('./unseen-data/node2vec_v2_data.csv.gz',
                         compression='gzip',
                         sep='\t')
    X_test = unseen.drop(columns=cols).values
    y_test = unseen[['label']].values
    model = emb_data([
        "is_dorm", "is_year", "year_diff", "from_high_school",
        "to_high_school", "from_major", "to_major", "is_faculty", "is_gender"
    ])
    yhat = model.predict(X_test)
    print(confusion_matrix(y_test, yhat))
    print(classification_report(y_test, yhat))
    print('ROC_AUC_SCORE:',
          roc_auc_score(y_test, model.decision_function(X_test)))
    print('MODEL SUMMARY:',model.intercept_, model.coef_)
    print('MODEL SUMMARY:',unseen.columns)   
    print(X_test[0])

In [55]:
def baseline_data():
    train = pd.read_csv('./data/baseline_train_filtered.data', sep='\t')
    test = pd.read_csv('./data/baseline_test_filtered.data', sep='\t')
    return LogReg(train,
                  test,
                  params={
                      'max_iter': 100000,
                      'solver': 'liblinear'
                  },
                  cols=[])

In [56]:
def unseen_base_data():
    unseen = pd.read_csv('./unseen-data/baseline_unseen_filtered.data',
                         sep='\t')
    X_test = unseen.drop(columns=['label']).values
    y_test = unseen[['label']].values
    model = baseline_data()
    yhat = model.predict(X_test)
    print(confusion_matrix(y_test, yhat))
    print(classification_report(y_test, yhat))
    print('ROC_AUC_SCORE:',
          roc_auc_score(y_test, model.decision_function(X_test)))
    print('MODEL SUMMARY:',model.intercept_, model.coef_)
    print('MODEL SUMMARY:',unseen.columns)   
    print(X_test[0])

In [57]:
def top_data():
    train = pd.read_csv('./data/topological_train_filtered.data', sep='\t')
    test = pd.read_csv('./data/topological_train_filtered.data', sep='\t')
    params = {
    'C': 0.001,  #100,
    'class_weight': None,
    'dual': False,
    'fit_intercept': True,
    'intercept_scaling': 1,
    'l1_ratio': None,
    'max_iter': 100000,
    'multi_class': 'auto',
    'n_jobs': None,
    'penalty': 'l1',
    'random_state': None,
    'solver': 'liblinear',
    'tol': 0.0001,
    'verbose': 1,
    'warm_start': False
}
    return LogReg(train,
                  test,
                  params=params,
                  cols=[])

In [58]:
def unseen_top_data():
    unseen = pd.read_csv('./unseen-data/topological_unseen_filtered.data',
                         sep='\t')
    X_test = unseen.drop(columns=['label']).values
    y_test = unseen[['label']].values
    model = top_data()
    yhat = model.predict(X_test)
    print(confusion_matrix(y_test, yhat))
    print(classification_report(y_test, yhat))
    print('ROC_AUC_SCORE:',
          roc_auc_score(y_test, model.decision_function(X_test)))
    print('MODEL SUMMARY:',model.intercept_, model.coef_)
    print('MODEL SUMMARY:',unseen.columns)   
    print(X_test[0])

In [59]:
unseen_base_data()

X: (65992, 3)
[[4960 3261]
 [  89 8188]]
              precision    recall  f1-score   support

         0.0       0.98      0.60      0.75      8221
         1.0       0.72      0.99      0.83      8277

    accuracy                           0.80     16498
   macro avg       0.85      0.80      0.79     16498
weighted avg       0.85      0.80      0.79     16498

ROC_AUC_SCORE: 0.940153589340453
[[ 6994  7083]
 [   89 13988]]
              precision    recall  f1-score   support

         0.0       0.99      0.50      0.66     14077
         1.0       0.66      0.99      0.80     14077

    accuracy                           0.75     28154
   macro avg       0.83      0.75      0.73     28154
weighted avg       0.83      0.75      0.73     28154

ROC_AUC_SCORE: 0.9263814594780213
MODEL SUMMARY: [-2.23955102] [[35.12658717 24.55445738  8.74795957]]
MODEL SUMMARY: Index(['jaccard', 'adamic adar', 'resource allocation', 'label'], dtype='object')
[1.65714286e-01 2.05347969e+02 4.81100000

In [60]:
unseen_emb_data()

X: (61867, 64)
[LibLinear][[9375  997]
 [1703 8548]]
              precision    recall  f1-score   support

           0       0.85      0.90      0.87     10372
           1       0.90      0.83      0.86     10251

    accuracy                           0.87     20623
   macro avg       0.87      0.87      0.87     20623
weighted avg       0.87      0.87      0.87     20623

ROC_AUC_SCORE: 0.9365540156119203
[[12680  1397]
 [ 2966 11111]]
              precision    recall  f1-score   support

           0       0.81      0.90      0.85     14077
           1       0.89      0.79      0.84     14077

    accuracy                           0.85     28154
   macro avg       0.85      0.85      0.84     28154
weighted avg       0.85      0.85      0.84     28154

ROC_AUC_SCORE: 0.9229366201819724
MODEL SUMMARY: [-93.21034384] [[3.67365969 4.00910664 2.74101355 4.37771168 4.38297982 4.06522574
  3.54705141 3.65401041 3.62687741 3.41746633 4.10202749 2.78892985
  4.80832674 4.90522543 3.27

In [61]:
unseen_top_data()

X: (65992, 9)
[LibLinear][[18261 14763]
 [  235 32733]]
              precision    recall  f1-score   support

         0.0       0.99      0.55      0.71     33024
         1.0       0.69      0.99      0.81     32968

    accuracy                           0.77     65992
   macro avg       0.84      0.77      0.76     65992
weighted avg       0.84      0.77      0.76     65992

ROC_AUC_SCORE: 0.9570162918177703
[[ 6507  7570]
 [   66 14011]]
              precision    recall  f1-score   support

         0.0       0.99      0.46      0.63     14077
         1.0       0.65      1.00      0.79     14077

    accuracy                           0.73     28154
   macro avg       0.82      0.73      0.71     28154
weighted avg       0.82      0.73      0.71     28154

ROC_AUC_SCORE: 0.9478721818457874
MODEL SUMMARY: [-0.58921265] [[0.         0.         4.58826673 0.         0.18697671 1.18756978
  0.         0.         0.03495417]]
MODEL SUMMARY: Index(['preferential', 'jaccard', 'adamic 